In [1]:
!pip3 install gpt-2-simple


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.8.1-py3-none-any.whl size=24576 sha256=a2c92eee011a00975cedbb77347364a31fe99be57d7451d2d2cdfa7f5a49366e
  Stored in directory: /root/.cache/pip/wheels/d6/89/8a/f5de6944286d1ac2658b0caa7eae3c8cda50f770cdc957217f
Successfully built gpt-2-simple


In [2]:
import numpy as np
import pandas as pd

## If Running in google colab
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
! ls /content/drive/MyDrive/CSE_272_Final_Project/


 artifacts
 chatbot.py
 checkpoint
 Covid-Utterances.xlsx
 CSE_272_Final_Presentation.gslides
 data
 Generative_Model_Evaluation.ipynb
 Generative_Model_Question_Answering.ipynb
 intents_model.ipynb
 IR_Project_Ideas.gdoc
 media
 models
 progress.mp4
'SlugDoc COVID Information Bot - 2 June 2022.mp4'
 System_Diagram.gslides


In [4]:
!cp -r /content/drive/MyDrive/CSE_272_Final_Project/checkpoint .

In [5]:
!cp /content/drive/MyDrive/CSE_272_Final_Project/data/COVID-Dialogue-Dataset-English.txt .


In [6]:
!cp /content/drive/MyDrive/CSE_272_Final_Project/data/generated_answers.txt .

In [6]:
import time

start = time.time()

file1 = open('COVID-Dialogue-Dataset-English.txt', 'r')
Lines = file1.readlines()
strings= []
strings_2 = []
for i, line in enumerate(Lines):
    if line.startswith('Description'):
        strings_2.append(Lines[i+1])
    if line.startswith('Patient:') or line.startswith('Doctor:'):
        strings.append(line)
        text = Lines[i+1]
        text = text.replace('Thanks for your question on Healthcare Magic.', ' ')
        text = text.replace('In brief:', ' ')
        text = text.replace('Would you like to video or text chat with me?', ' ')
        text = text.replace('Thanks for using Healthcaremagic.com', ' ')
        text = text.replace('Respected user', ' ')
        strings.append(text)

f = open("questions_and_responses.txt", "w")
for i in range(len(strings)):
    f.write(strings[i].strip('\n'))
    f.write('\n')
end = time.time()
print(end-start)


0.011346101760864258


In [18]:
strings

['Patient:\n',
 'Hello doctor, I get a cough for the last few days, which is heavy during night times. No raise in temperature but feeling tired with no travel history. No contact with any Covid-19 persons. It has been four to five days and has drunk a lot of Benadryl and took Paracetamol too. Doctors have shut the OP so do not know what to do? Please help.\n',
 'Doctor:\n',
 'Hello, I understand your concern. I just have a few more questions.\n',
 'Patient:\n',
 'Thank you doctor,\n',
 'Doctor:\n',
 'Hi, I would recommend you take n-acetylcysteine 200 mg powder dissolved in water three times a day. You may also nebulize using PNSS (saline nebulizer) three times a day. This will help the phlegm to come out. I would also recommend you take vitamin C 500 mg and zinc to boost your immune system. If symptoms persist, worsen or new onset of symptoms has been noted, further consult is advised.\n',
 'Patient:\n',
 'Hello doctor, I am suffering from coughing, throat infection from last week. A

## Standard Metrics

In [9]:
patient_questions = []
doctor_responses = []
for i, val in enumerate(strings):
    if val.startswith('Patient:') and strings[i+2].startswith('Doctor'):
        if strings[i+1].startswith('Thank you doctor'):
            continue
        patient_questions.append(strings[i+1])
        doctor_responses.append(strings[i+3])


In [10]:
import gpt_2_simple as gpt2

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)
answers = []
for question in patient_questions[:100]:
    pre = 'Patient:\n {} Doctor:\n '.format(question)
    res_txt = gpt2.generate(sess,
                            length=30,
                            temperature=0.1,
                            prefix=pre,
                            nsamples=1,
                            batch_size=1, 
                            return_as_list=True
                            )
    answers.append(res_txt[0].split('Doctor:\n ')[1])
answers

Loading checkpoint checkpoint/run1/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-2000


KeyboardInterrupt: ignored

In [17]:
patient_questions[:5]

['Hello doctor, I get a cough for the last few days, which is heavy during night times. No raise in temperature but feeling tired with no travel history. No contact with any Covid-19 persons. It has been four to five days and has drunk a lot of Benadryl and took Paracetamol too. Doctors have shut the OP so do not know what to do? Please help.\n',
 'Thank you doctor,\n',
 'Hello doctor, I am suffering from coughing, throat infection from last week. At that time fever did not persist and also did not felt any chest pain. Two days later, I consulted with a doctor. He prescribed Cavidur 625, Montek LC, Ambrolite syrup and Betaline gargle solution. Since then throat infection improved and frequent cough also coming out. Coughing also improved remarkably though not completely. From yesterday onwards fever is occuring (maximum 100-degree Celcius). I have not come in touch with any foreign returned person nor went outside. In our state, there is no incidence of Covid-19. Please suggest what to

In [15]:
answers[:5]

['  Cough of COVID-19.   Follow the following guidelines:  .\n\n\nPatient:\nI have been sick for going',
 '  This could well be very normal.   One can get pain on day 14 during ovulation and before and during the period. Use panado',
 ' In my opinion, you should get done a chest x-ray and PFT (Pulmonary Function Test). A chest x-ray is needed',
 '  This could well be very normal.   One can get pain on day 14 during ovulation and before and during the period. Use panado',
 '  This could well be very normal.   One can get pain on day 14 during ovulation and before and during the period. Use panado']

In [16]:
doctor_responses[:5]

['Hello, I understand your concern. I just have a few more questions.\n',
 'Hi, I would recommend you take n-acetylcysteine 200 mg powder dissolved in water three times a day. You may also nebulize using PNSS (saline nebulizer) three times a day. This will help the phlegm to come out. I would also recommend you take vitamin C 500 mg and zinc to boost your immune system. If symptoms persist, worsen or new onset of symptoms has been noted, further consult is advised.\n',
 'Hello, I can understand your concern.\n',
 'Hi, yes, upload in this query only. I will see and revert to you.\n',
 'Hi, I can understand your concern. I have gone through the report you have attached. (attachment removed to protect patient identity).\n']

In [13]:
import re
import string
documents_clean = []
for d in doctor_responses[:97]:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)


In [14]:
answers_clean = []
for a in answers:
    # Remove Unicode
    answer_test = re.sub(r'[^\x00-\x7F]+', ' ', a)
    # Remove Mentions
    answer_test = re.sub(r'@\w+', '', answer_test)
    # Lowercase the document
    answer_test = answer_test.lower()
    # Remove punctuations
    answer_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', answer_test)
    # Lowercase the numbers
    answer_test = re.sub(r'[0-9]', '', answer_test)
    # Remove the doubled space
    answer_test = re.sub(r'\s{2,}', ' ', answer_test)
    answers_clean.append(answer_test)


In [21]:
!pip install -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 4.2 MB/s 
     |████████████████████████████████| 749 kB 69.7 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [56]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score
import nltk
nltk.download('wordnet')
bleus = []
meteors = []
for (ref, cand) in zip(documents_clean, answers_clean):
    reference = ref.split(' ')
    candidate = cand.split(' ')
    bleus.append(sentence_bleu(reference, candidate))
    meteors.append(single_meteor_score(reference, candidate))
print(bleus)
print(meteors)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  """


[0.5081327481546147, 0, 0.4566337854967312, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.4428500142691474, 0, 0, 0, 0.5555238068023582, 0, 0.6262844962765469, 0.4566337854967312, 0.5695988432761473, 0.5555238068023582, 0, 0.4518010018049224, 0.4566337854967312, 0.5623413251903491, 0.4518010018049224, 0, 0, 0.4518010018049224, 0, 0, 0.4671379777282001, 0, 0, 0, 0, 0.447213595499958, 0, 0.5946035575013605, 0.47897362544357464, 0, 0, 0, 0, 0, 0, 0.4728708045015879, 0, 0, 0, 0, 0, 0.5169731539571706, 0, 0, 0, 0, 0.4428500142691474, 0.4671379777282001, 0.5430321466633391, 0.447213595499958, 0, 0, 0.46173663094410267, 0.4566337854967312, 0.4566337854967312, 0.4518010018049224, 0, 0, 0, 0.46173663094410267, 0, 0.5695988432761473, 0.5318295896944989, 0.4566337854967312, 0.447213595499958, 0, 0.5430321466633391, 0.5828233954152654, 0, 0, 0.447213595499958, 0, 0.5169731539571706, 0, 0, 0.4518010018049224, 0, 0, 0.47897362544357464, 0.4566337854967312, 0.4428500142691474, 0.47897362544357464, 0, 0]
[0.1136363

In [57]:
np.mean(bleus)

0.20738822519707822

In [58]:
np.mean(meteors)

0.11697938227149868

In [40]:
!pip install rouge-metric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 151 kB 4.1 MB/s 


In [52]:
from rouge_metric import PyRouge
rouge = PyRouge(rouge_n=(1, 2), rouge_l=True, rouge_w=False, rouge_s=False, rouge_su=False)
rouges = []
for (ref, cand) in zip(documents_clean, answers_clean):
    reference = ref.split(' ')
    candidate = cand.split(' ')
    reference = [val for val in reference if val != '']
    candidate = [val for val in candidate if val != '']
    if len(reference) > len(candidate):
        reference = reference[:len(candidate)]
    elif len(candidate) > len(reference):
        candidate = candidate[:len(reference)]
    rouges.append(rouge.evaluate_tokenized(candidate, reference))
rouges

[{'rouge-1': {'f': 0.0954499275098951,
   'p': 0.055916005291005294,
   'r': 0.3257936507936508},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.0954499275098951,
   'p': 0.055916005291005294,
   'r': 0.3257936507936508}},
 {'rouge-1': {'f': 0.12878601738426879,
   'p': 0.08235930735930738,
   'r': 0.2951839826839827},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.12878601738426879,
   'p': 0.08235930735930738,
   'r': 0.2951839826839827}},
 {'rouge-1': {'f': 0.05901475809054366,
   'p': 0.03531746031746031,
   'r': 0.17936507936507937},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.05901475809054366,
   'p': 0.03531746031746031,
   'r': 0.17936507936507937}},
 {'rouge-1': {'f': 0.08333762113026327,
   'p': 0.052314814814814814,
   'r': 0.20476190476190476},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.08333762113026327,
   'p': 0.052314814814814814,
   'r': 0.20476190476190476}},
 {'rouge-1': {'f': 0.10125

In [67]:
rouge_1 = []
ps = []
rs = []
for dic in rouges:
    rouge_1.append(dic['rouge-1']['f'])
    ps.append(dic['rouge-1']['p'])
    rs.append(dic['rouge-1']['r'])

In [68]:
np.mean(ps)

0.08125567488392318

In [69]:
np.mean(rs)

0.3202228497050191

In [70]:
np.mean(rouge_1)

0.12763891309948203

## Custom Metric

In [ ]:
# Warning-- takes a long time to load and can crash your runtime!
import gpt_2_simple as gpt2

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)
answers = []
for string_ in strings_2[:100]:
    pre = 'Patient:\n {} Doctor:\n '.format(string_)
    res_txt = gpt2.generate(sess,
                            length=50,
                            temperature=0.1,
                            prefix=pre,
                            nsamples=1,
                            batch_size=1, 
                            return_as_list=True
                            )
    answers.append(res_txt[0].split('Doctor:\n ')[1])

answers

Loading checkpoint checkpoint/run1/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-2000


['  Source of pneumonia   Unlikely source of pneumonia, but management typically best based on the person’s history and symptoms.   Sometimes you have to answer to a lot more than just your average doctor.      ',
 '  High    or possibly COVID-19, likely flu, or covid 19.   Call your local doctor to discuss these new changes in your health to determine what testing will,be needed, and steps you should take. ',
 '  Yes    Yes, but I would not recommend it for any reason.   Throat pain can be from a strep throat infection (antibiotics needed), a cold or influenza, or from some other cause such as allergies or',
 '  It could   There is not enough tests in the world right now to do it, and that is a big problem given the hysteria in the US.   The hysteria is so bad that one was hospitalized one week ago for it. One',
 '  If you are living in an area affected with covid-19 you would probably://t.co/fVfVf2e2eI    and stay away from hospitals, and follow OMS guidelines    ',
 '  Eradication  

In [ ]:
f = open("generated_answers.txt", "w")
for i in range(len(answers)):
    f.write(answers[i].strip('\n'))
    f.write('\n')



In [ ]:
!cp generated_answers.txt /content/drive/MyDrive/CSE_272_Final_Project/

In [8]:
file1 = open('COVID-Dialogue-Dataset-English.txt', 'r')
Lines = file1.readlines()
strings= []
strings_2 = []
count = 0
current_pos = 0
doctor_responses = []
for i, line in enumerate(Lines):
    if i<current_pos:
        continue
    # if count > 107:
    #     break
    if line.startswith('Description'):
        count += 1
        current_pos = i+1
        string = ''
        while (current_pos<len(Lines)) and not Lines[current_pos].startswith('Description'):
            if Lines[current_pos].startswith('Doctor:'):
                sub_count = 1
                while (current_pos+sub_count<len(Lines)) and not (Lines[current_pos+sub_count].startswith('Patient:') or Lines[current_pos+sub_count].startswith('id')):
                    string += Lines[current_pos+sub_count]
                    sub_count+=1
            current_pos+=1
        doctor_responses.append(string)


In [9]:
len(doctor_responses)

604

In [10]:
!ls

checkpoint			    generated_answers.txt
COVID-Dialogue-Dataset-English.txt  questions_and_responses.txt
drive				    sample_data


In [ ]:
file1 = open('generated_answers.txt', 'r')
Lines = file1.readlines()
answers= []
for line in Lines:
    answers.append(line)


In [ ]:
answers = [val for val in answers if val != '\n']
len(answers)

99

In [11]:
answers = ['  Source of pneumonia   Unlikely source of pneumonia, but management typically best based on the person’s history and symptoms.   Sometimes you have to answer to a lot more than just your average doctor.      ',
 '  High    or possibly COVID-19, likely flu, or covid 19.   Call your local doctor to discuss these new changes in your health to determine what testing will,be needed, and steps you should take. ',
 '  Yes    Yes, but I would not recommend it for any reason.   Throat pain can be from a strep throat infection (antibiotics needed), a cold or influenza, or from some other cause such as allergies or',
 '  It could   There is not enough tests in the world right now to do it, and that is a big problem given the hysteria in the US.   The hysteria is so bad that one was hospitalized one week ago for it. One',
 '  If you are living in an area affected with covid-19 you would probably://t.co/fVfVf2e2eI    and stay away from hospitals, and follow OMS guidelines    ',
 '  Eradication   Stay at home, monitor your temperature, drink fluids and rest.   Nasal congestion and post nasal drip is not typically associated with COVID-19. Watch for dry cough, fever and if your shortness of',
 '  Probably not COVID.   A person who gets pneumonia does not show any symptoms such as the cough, fever, or shortness of breath.   The cough is usually gone within 30 days. The shortness of breath is usually gone',
 '  Can talk with doctor   Doctors can give you the up-to-date information.   Video consultations available (start over and click on Talk-to-Doctor). We have informational posts on our Healthtap.com/blog. As',
 "  Unknown   Without any details it's impossible to say.         \n\n\nPatient:\nMy hba1c is not as good as it should be. Am I more likely to die of the",
 '  Yes vomiting and diarrhea would be fewer after 19.  E.C.B.R.D. is the number one cause of pneumonia in patients receiving antibiotics, and the remaining infection is spread through respiratory route.   Only severe infections are',
 '  Unknown.   Because I am female, I am at greater risk for getting it.   https://www.healthtap.com/blog/covid-19-care-guidelines/self-quarantine-guide.',
 '  It is not    like the covid-19s and how long ago this illness was endemic.   It is more like a mild respiratory inflammatory condition.   The COVID-19 infection is worldwide. The goal is to',
 '  Yes   yes, get tested, but you should.   If you get mild symptoms, including fever, dry cough and/or shortness of breath, call your health care provider.   If you are otherwise healthy, self-',
 '  Consult with your GP   You cannot just request a COVID19 test, without fulfilling certain official SA testing criteria, even if you are a private patient and are willing to pay cash.  If you meet the strict testing criteria,then your',
 '  High risk country   The US is one high risk country for COVID19.   The following table details risk factors for death from the virus:   US birth weight, older, younger than 70, both males and females, both',
 '  Test if can detect covid-19   Is it a viral or bacterial infection? If you have a fever and/or the cough or shortness of breath you should test if you have it.  If you have no symptoms, you',
 '  Cough with a sore throat   The cough with a sore throat is consistent with an allergic reaction.   There are other possible causes for this, including an allergic reaction, but allergen induced, as is the case with COVID',
 '  HArd to say   If you have access to a telephonic consult then rather do that to start.   If you have access to testing you can do that. Do you have to work at the moment? I would discuss',
 '  Data   From the beginning the feeling was with bronchitis there.   And then one got mild symptoms and went back to the hospital, the other two started feeling better. Follow that with fever and you have a clear picture of what',
 '  diarrhea   If you are having no diarrhea you are fine.   If you have fever, you are better to be tested.    If you have cough and shortness of breath, you are more at risk of getting the virus',
 '  TRANSMISIÓN VIOLENCE   The virus causes a person to penetrate a person syndrome (sexually transmitted infection).   A symptomatic patient with secondary infection may be de-isolated 14 days after achieving clinical isolation',
 '  YEs   This virus clings to everything especially hairs.   Either wash your beard a few times a day and spray with alcohol or shave off.           \n\n\nPatient:\n',
 '  COVID-19 and Asthma   Very few people recover from a corona virus attack, so it is extremely important to self-quarntine.   However, the symptoms of COVID-19 are often the same as',
 '  Consider alternative   SEeoxgen availability and screening is limited, and your patient may be more susceptible to Covid-19 complications.           \n\n\nPatient:\nI use Athra',
 "  Odd comment. The way the WHO says so.   In all cases, follow the recommendations of the department of respiratory and respiratory diseases and the way to be vaccinated is to self-quarntine. Don't worry.   ",
 '  No    if you do not have a fever, then you are not infected.   but if you have a cough, then it is likely that you are.           \n\n\nPat',
 '  TRANSMISIÓN VIOLENCE   The virus causes a person to penetrate a person syndrome (sexually transmitted infection).   A symptomatic patient with secondary infection may be de-isolated 14 days after achieving clinical isolation',
 "  New data   From my perspective, I don't think it. People get nauseaus, and I don't think they necessarily mean flu- rather, they mean:\n- A sudden onset of cough and mild cough\n- High likelihood of",
 '  Hard to say\n   Most of the tests cover the globe.   The questions you pose are more appropriate to ask your local physicians and health plans.           \n\n\nPatient:\n',
 '  Travel is a risk   Any travel is a risk, that is how this has spread so yes I would advise you telephonically consult with a doctor and see what precautions arein place to prevent catching this disease.   But locking',
 '  Vitamin C   If you are getting symptoms of corona, you should get tested for Coronavirus.    If you are getting symptoms of flu, corona virus may be present. Should get tested.\n\n\nPatient',
 '  Yes!    I am sure that will help you.          \n\n\nPatient:\nLast year during flu season had a severe cough, difficulty breathing and xray show there was fluid in',
 '  Yes and no.   Yes if not done correctly, can cause whiteout and also infection.ilation.    This is very normal. If not, it can be seen almost everywhere. Hope I have answered your query. Let me',
 '  Yes   Yes cause urine to be drawn and if so how much is enough for HIV.   It is not a substitute treatment.          \n\n\nPatient:\nWhat are the chances of',
 '  No.   Cough of pneumonia is typically management pneumonia.   In this case, blood clot in the lung is reassuring but may not be immediately evident. In addition, if fever does emerge, it is likely that this was caused by',
 '  Wait for more symptoms   This can take up to 14 days after the infection.   Inhaled bronchodilators are the right choice of cure for pneumonia because the virus will generally not survive longer in the body than in another',
 '  Inhaled bronchodilators   2-3 weeks   Inhaled bronchodilators are best for pneumonia because of the mucosal damage done by bronchitis.   Inhaled bronchodil',
 '  Wrong place to do so   In my opinion, you should consult a doctor rather than go to a hospital.   You should get screened for pneumonia, since screened tests can be dangerous. Your best advice would be to do a chest x',
 "  Infection   When you have the infection, it's usually not more than a few hours at a time.   If your symptoms get worse, you should be seen by your doctor. If you've been exposed to a particular illness,",
 '  Pneumonia   In pneumonia patient has chest x ray and PFT (Pulmonary Function Test).   PFT is needed to rule out lung infection. If still worried about the infection,oria, and possible pneumonia,be kind',
 "  Probable   Inhaled bronchodilators are a good treatment for pneumonia.   But the drugs are being discontinued because of shortages. So you won't be seeing them in the US for medical reasons. In any event,",
 '  Treatment for pneumonia.   By taking pneumococcal and influenza vaccine, one is less at risk of developing pneumonia.   However, one needs to be cautious when it comes to pneumonia. In the US, one should first discuss with your',
 '  Pneumonia   Most of pneumonia is bacterial, so better to consult pulmonologist and get done sputum culture and sensitivity report.  Culture report will isolate the causative organism (if it is MRSA or other bacteria) and',
 '  Notwise not to do it.   Develop weight loss symptoms, fatigue and dry cough.Can call doctor by video or phone.Good luck.           \n\n\nPatient:\nMy 5',
 '  Viral respiratory    The virus causes bronchospasm in the bronchi, bronchiectasis (inflammation of airways) in the lung, and the pneumonia causes bronchospasm in the bronchiectesis. ',
 '  Wait 2-4 weeks   Wait for the culture to identify the causative organism, if it is, it is not treatable with antibiotics.   Get done sputum culture and sensitivity report. Culture will identify the causative organism',
 "  PTB   If you don't have it, don't need to worry.   But if you do have it, it can help you to relax, reduce stress, and improve immunity.   Sleep apart... your call! ",
 '  Treatment   One treatment for pneumonia is supportive antibiotic therapy.   One can receive supportive therapy for pneumonia, supportive therapy for bronchitis, and supportive therapy for pneumonia. So one can receive bronchodilator medication and inhaled bron',
 '  The treatment   Start by talking with your GP.   If you are getting better, you may be able to start taking paracetamol.   If not, it is another option. If you are better, you can continue',
 '  Repeated chest CT findings show there was pneumonia but treatment was to be speedy.   Inhaled bronchodilators are another treatment option for bronchitis.    But dehornodilators have been reported before,',
 '  Per RSA NICD:   Seek care right away if difficulty breathing, other severe symptoms.  Seek testing if sick, PLUS known COVID-19 contact, or travel to high risk country, or been in facility treating people with COVID',
 "  Per RSA NICD:    You should give up smoking as it causes a multitude of problems and may well be related to the infection.   Don't worry, you will be alright with all these. Hope I have solved your query",
 '  Topical drainage   This is more likely in younger patients.   However, the absence of fever and the cough with or without fever should be considered mild symptoms of pneumonia.    Anorexia will usually resolve with  clinical description',
 '  Wait for after CHP CT   Wait for after you and your partner is completely off the meds, or if you are just fine and are resting.   We are here for you.         ',
 '  Antibiotics   Antibiotics are the answer to getting better.   Get as much pain as you can into your ribcort and keep hurting it. Pain is not with numbers and gory things can be done at this time',
 '  Yes   Yes cause pain, discomfort and make you more vulnerable to infections.   A report from the United States indicated that you up-regulated the PNAP1 gene. This gives you a gene that is highly expressed in the lung',
 '  Yes and no.   Yes would not be taking the Nyquil with steroids.   It was discovered that the Nyquil with steroids was having an extremely low rate of recurrence. Antibiotics are the number onestay of',
 '  Yes and no.   Yes for the pneumonia is severe, but within the group of conditions under ERTI (Emergency Room) treatment.   Treatment is not exact and typically willroximate studies. Trials are being done in France, Germany',
 '  Fever   Chest infection can cause asthma, bronchitis and bronchitis.   Inhaled bronchodilators are the first line treatment for bronchitis.    Bilateral abscess is often not treated with',
 ' Levofloxacin is generally advised in adults.  I think the younger the patient, the better the immune system response. So better to advise by video instead of requiring an in-person visit. Use good hand washing and keep surfaces clean',
 '  Tummy Bug   Because of the mucosal effects of the bronchitis on the lung, I would not recommend taking steroids for lung symptoms.   Also, the deficiences of the disease do not persist after lung treatment.  ',
 '  Not stablished.   The pain in the chest region is not in relation with pneumonia.   Call your General Physician and check if they want to see you.   You may need ventilator treatment and further management. ',
 '  Fever and cough   The expression of pneumonia is controlled.   The cough usually goes away in 2-3 days. The expression of pneumonia is being monitored closely. If you have a fever and cough, please call your PCP right away',
 '  May do video consult   Throat pain can be from a strep throat infection (antibiotics needed), a cold or influenza, EB virus, or from some other cause such as allergies or irritants.   Usually, a person',
 '  Yes    if you are taking Imodium during the Naproxen Naproxen cycle you will be well for clinical diagnosis.   If you are not taking either Naproxen or Claritromycin start taking these medicines. ',
 '  No   In pneumonia, the bronchi is the major cell in pneumonia and the bronchiase is responsible for the cough, not the other way around.   Inhaled bronchodilators and inhaled corticosteroid',
 '  Wrong place to ask question   This is the HT public site where volunteer docs primarily in the USA answer health questions.   We do not do referrals/provide notes or prescriptions or any other form of direct service. You need to contact',
 '  Tracts, muscles, and/or organs affected by pneumonia.   Usually, a person sees the doctor if the situation requires urgent treatment. The chest pain can be from a strep throat (antibiotics needed), a cold or',
 '  Yes and no.   Yes if I m promptly and fairly as I m now, but still maintaining a level of health that is above par.   No to being well but very much in the running. I need some advice please?',
 '  Amount of cells affected    Minimal amount of cells that are affected is Minimal because your symptoms are getting worse.   In my opinion, you should should start taking acetaminophen or ibuprofen. Ibuprofen',
 '  Pneumonia   Using CPAP machine for pneumonia is extremely prone to death.   The best thing you can do for yourself is to stay home, rest and drink fluids.   You may benefit from counseling from a clinical psychologist via',
 '  Same same same   You have the same symptoms, just you have pneumonia.   Pepcid is the same, I believe you could get the same. I have gone through your doctor before, and it was the other way. I',
 '  Pneumonia.   Usually, the chest infection is in the airways not in the bronchi. But sometimes, the pain is not in the lungs and the pain is not in the throat. The chest infection takes 2-3 weeks',
 "  YEs   This can be anything from a cold or influenza to a sore throat.   Usually, a person sees the doctor if the sore throat is bothersome, recurrent, or doesn't go quickly (mainly to check for stre",
 '  Doxycycline   This may be a precursor to this in your case.   Follow the following guidelines: https://www.healthtap.com/blog/covid-19-care-guidelines/preventing',
 '  Wait for after pneumonia.   Wait for after pleurisy after influenza or coronavirus infection.   Take levocetrizine, paracetamol, and Z-Pak. Do warm water gargles and steam inhalation',
 "  No   while having a bronchitis or pneumonia your symptoms are more suggestive of pneumonia.   In my experience most type 1's do not transmit bronchitis or pneumonia.   If: - you have traveled to a high risk",
 '  Systemic infection   Strep infection, Influenza, or another infection may be at play.   Inhaled bronchodilators may help prevent bronchitis.           \n\n',
 '  Primary risk   If you are followed correctly you should not have to worry about your health.   Follow the guidelines all over the media and make sure your colleagues are aware. The risk of catching pneumonia with smoking is very low.  ',
 '  Clarify chronic illness   By visiting a hospital emergency room, attending a pulmonologist, or completing a clinical prescription, one can be presumed to be in good control of your symptoms.   You should also monitor your temperature and avoid environmental',
 '  Cough of pneumonia   This can be either bacterial pneumonia or a viral infection.   In both cases, health plans should follow CDC guidelines on this.  Find a Dr online and chat with them. Stay home.    ',
 '  Yes:   Not wheezing,for chest pain and nausea ( nausea when coughing and feeling sick, preferably with pneumonia or tuberculosis) it is very normal.   But breathing difficulties can be from a strep throat infection (antib',
 "  No   There is not enough data to support it's use.   More clinical trials are needed           \n\n\nPatient:\nI need to check if I have COVID-19",
 '  Maybe\n   This can be in any of the following:   Malignancy of the lung, pretreatment of which will not be solved with antibiotics.   Inhaled bronchodilators are the next best thing.',
 '  Yes and no.   Yes frequent nose bleeds and sever cough signs be reported for Coronavirus.    If your symptoms are worsening or your cough is also worsening, notify your provider and get tested.  If your symptoms',
 '  MAke embolism your husband had pneumonia in the pneumonia department?   Yes i agree with the treatment its ok continue it as the doctor gad prescribed it, its ok continue it as the patient , its ok continue it as the doctor ,',
 '  Best to be treated by video.   Inhaled bronchodilators and inhaled corticosteroid (ICS) are the mainstay of treatment for bronchitis. Inhaled bronchodilators help to relieve',
 '  Not really   It is more associated with the symptoms of pneumonia.   You have been treated with radiological medicine and there is no definitive test for it.           \n\n\nPatient:',
 '  Best   Best management depends upon the cause of death; however, the incidence and extent of infection is not yet well controlled.   The following guide should help: - Patients           \n\n\n',
 '  Wait 2-4 weeks   Wait for after pneumonia.   Can have complication of coronavirus infection, such as pneumonia or tuberculosis.   Can also get bronchitis or flu. Should get done trach.\nPatient',
 '  No    antibiotics are the right drugs and I think they will do the treatment for pneumonia.   But the Amox is a new class of drugs and there are many more that we can take in tandem with antibiotics for pneumonia. So',
 '  Inhaled bronchodilators may help in pneumonia.   Inhaled bronchodilators may help in bronchitis. And in either of those cases, treatment is beginning. So start it on. Repeated bron',
 "  Same as above   Inhaled bronchodilators are for pneumonia.   Inhaled bronchodilators are for bronchitis. And they don't protect you against COVID-19.    ",
 '  Systemic infection   Pneumonia is less likely to cause public health concern, because pneumonia does not cause lung infection.   But what you see is more likely. If you have traveled to endemic areas or met with people that traveled there',
 '  Fever with phlegm expectoration   Call your provider and or health department.   In the mean time, self quarantine and monitor your temperature.           \n\n\nPatient:\nMy',
 "  If you are already having a cough and don't have a fever, don't go to a GP.   If you do have a cough, get tested. The test will identify who you are and who should be tested, based on the",
 '  Maynard infection   Bypassing through nose through not getting in the way, doctor can get through to x ray.   Strep infection, influenza are possible infections. Antibiotics are the mainstay of treatment for pneumonia. In',
 '  Tummy infection   Antibiotic is being used in a large scale community acquired pneumonia (MAC). The goal is to prevent infection of patients with respiratory disease into the body. To do this, the doctor should give abridaged treatment',
 '  No headaches.   YOur office is here.   Call today (03/22/20) and answer the following           \n\n\nPatient:\nHave had a fever since the',
 '  Yes and no.   Yes if not, it is not contagious and testing is done after the cough or the sore throat.   If you are otherwise healthy, await test results. Keep comfy in a busy office. Also, if']
len(answers)

100

In [12]:
import re
import string
documents_clean = []
for d in doctor_responses:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)



In [13]:
answers_clean = []
for a in answers:
    # Remove Unicode
    answer_test = re.sub(r'[^\x00-\x7F]+', ' ', a)
    # Remove Mentions
    answer_test = re.sub(r'@\w+', '', answer_test)
    # Lowercase the document
    answer_test = answer_test.lower()
    # Remove punctuations
    answer_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', answer_test)
    # Lowercase the numbers
    answer_test = re.sub(r'[0-9]', '', answer_test)
    # Remove the doubled space
    answer_test = re.sub(r'\s{2,}', ' ', answer_test)
    answers_clean.append(answer_test)


In [32]:
strings_2[3]

'I have chills, breathing problems and cough with white phlegm. Could it be Coronavirus infection?\n'

In [31]:
answers_clean[3]

' it could there is not enough tests in the world right now to do it and that is a big problem given the hysteria in the us the hysteria is so bad that one was hospitalized one week ago for it one'

In [30]:
documents_clean[3]

'hello please answer the following any travel history in the last one month any use of airports any fever have you checked the temperature any cough any mucus or phlegm how is your sleep is there any malaise or body aches or weakness hi most probably with the history it does not appear like a severe problem or coronavirus infection is sore throat severe and does the pain last the whole day do you smoke or drink can you try taking sudafed tylenol and zyrtec and try getting sleep for a few days i am sure that will make you feel better avoid exposure to smoke dust and cold winds keep the ac to a minimum or use ceiling fan instead drink plenty of water and warm fluids avoid eating out partying avoid unnecessary travel you need not quarantine yourself in the strictest manner but some amount of restrictions in interacting and meeting with the community is advised lastly where are you currently your place of residence and country is it a corona endemic country did you talk to or interact with

In [ ]:
# Turn docs in tfidf vectors
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents_clean)
X = X.T.toarray()
df = pd.DataFrame(X, index=vectorizer.get_feature_names())


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,594,595,596,597,598,599,600,601,602,603
aasthma,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
abdominal,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
abgfor,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
ability,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
able,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zantac,0.00000,0.0,0.0,0.041084,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
zealand,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
zinc,0.11462,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
zulu,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.189976,0.0,0.0


In [ ]:
def get_similar_responses(q, df):
    q = [q]
    q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    sim = {}
    for i in range(df.shape[1]):
        sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
    sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
    return sim_sorted
q1 = answers[0]
print(get_similar_responses(q1, df))


[(575, 0.36578417376662326), (154, 0.1980102039259189), (471, 0.1692508612170635), (136, 0.1644319045330726), (362, 0.1632774445975073), (134, 0.14480673702621083), (440, 0.1440067591510045), (111, 0.14393219874428703), (487, 0.1414221744241368), (41, 0.1410141857086122), (114, 0.13494528995932437), (159, 0.13240383023934932), (68, 0.13053994351327772), (71, 0.1302442238959661), (124, 0.13007649779243316), (150, 0.12974148292215296), (82, 0.12877522818776282), (510, 0.12823063072626095), (365, 0.12771624377722696), (318, 0.1260568221011443), (172, 0.12597410878407825), (3, 0.1245371081416671), (208, 0.12417882560565828), (497, 0.12313136101776813), (5, 0.12302373663142319), (491, 0.12302373663142319), (67, 0.12272728007343695), (528, 0.11861849230697219), (151, 0.11843216368204475), (147, 0.1172837092662219), (490, 0.1172420913849322), (100, 0.11699754403054656), (138, 0.11629982330695947), (373, 0.11531915170141897), (105, 0.11471100209669519), (62, 0.11413363371466112), (54, 0.113752

In [ ]:
thing = [j[0] for j in get_similar_responses(q1, df)] 
np.where(np.asarray(thing) == 0)[0]

array([161])

In [ ]:
hits = 0
where_hit = []
for i, answer in enumerate(answers):
    thing = [j[0] for j in get_similar_responses(answer, df)] 
    where_hit.append(np.where(np.asarray(thing) == i)[0])
    top_10_inds = thing[:10]
    if i in top_10_inds:
        hits += 1
print('recall: {}'.format(hits/len(answers)))

recall: 0.12


In [ ]:
recall_thresholds = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
recall_vals = []
for val in recall_thresholds:
    recall_vals.append(sum([j<val for j in where_hit])/100)

In [ ]:
recall_vals

[array([0.1]),
 array([0.12]),
 array([0.14]),
 array([0.16]),
 array([0.18]),
 array([0.2]),
 array([0.22]),
 array([0.26]),
 array([0.28]),
 array([0.29]),
 array([0.3]),
 array([0.31])]

In [ ]:
answers[0]

'   Unlikely covid.   Unlikely to be corona at this time of year. Best to stay home and rest :) wash your\n'

In [ ]:
doctor_responses[373]

'Unlikely covid. Unlikely to be corona listening to your symptoms. Stay home and wear masks (anything that covers nose and mouth) proven to decrease transmission by 50%. If you are unwell in terms of respiratory rate (google) >25 or become medically unstable you need to find a corona hospital emergency room. Outside of these conditions stay home and stay safe. The more we all travel in the next 3 days is bad.\n\n'